# 3. Analyze WhatsApp Data

By: Sharon Makunura

In [8]:
import re
import pandas as pd
import numpy as np
import regex
from wordcloud import WordCloud , STOPWORDS , ImageColorGenerator
from nltk import *


In [9]:
whatsapp_df=pd.read_csv('whatsapp_dataset1.csv')

In [10]:
whatsapp_df.head()

,Date,Time,Sender,Chat
0,2021-01-01,00:02,GC6,media shared
1,2021-01-01,00:02,GC2,happy new year my beloved family
2,2021-01-01,00:03,Uncle,happy happy all
3,2021-01-01,00:04,Uncle1,happy new year
4,2021-01-01,00:09,Aunt 4,happy new yearrrrrr


In [20]:
#set date as date object
whatsapp_df['Date'] = pd.to_datetime(whatsapp_df['Date'])

In [117]:
#function to convert string series into list, takes series, returns list 
#null values should equal 'Empty

def stringSeries(a_series):
    stringed_s=[s for s in a_series if s!='Empty'] 
    return stringed_s


#redo above function but taking in multiple columns
#cannot work with emoji text as it reads multiple emojis as a different string
#will also return empty as a unique value

def stringCols(a_df):
    stringed_c=np.unique(a_df.values).tolist()
    return stringed_c

In [119]:
#function to get unique values in list of strings; returns list and integer
#function uses numpy

def listStrU(a_list):
    array_list = np.array(a_list)
    unique_list=np.unique(array_list).tolist()
    x=len(unique_list)
    return unique_list,x
    
    

In [123]:
members=whatsapp_df.Sender.unique()

In [11]:
#create functions to find most active days 
#references previous functions


def activeDay(a_series):
    most_days=a_series.mode()
#deal with results where more than one day had highest score  
    if len(most_days)>1:
#resultant object will be series, so convert to string,then join up resultant list into a string
        days_freq=stringSeries(most_days)
        most_freq_days=', '.join(map(str,days_freq))
    else:
#assign value at 0 to variable, to avoid printing systems info
        most_freq_days=most_days[0]
    
    return most_freq_days
        
        

In [12]:
#create functions to find most active hours
#references previous functions


def activeHour(a_tseries):
#begin by splitting column into hour[0] and minutes[1]
    split_series = a_tseries.str.split(':', expand=True)
    hours = split_series[0]
#find most popular hour
    most_hour=activeDay(hours)
    #if len(most_hour)>1:
        
    return most_hour

In [13]:
def wordCounter(a_wordSeries):
    index_count=a_wordSeries.apply(lambda w : len(w.split()))
    #adjust to include the actual messages
    high_c=index_count.max()
    low_c = index_count.min()
   
    all_c = index_count.sum()
    ave_c = round (all_c/len(a_wordSeries),2)
    return all_c,high_c,low_c,ave_c
    
    
    

In [14]:
#create function to replace some text in messages that should not be counted as text
def txtMsg(msg_series):
    non_msg_dict={' media shared':"",' this message was deleted':""}
    txt_series=msg_series.replace(non_msg_dict)
    
    return txt_series


In [15]:
#create function to count number of words in messages 

def freqWord(msg_series):
    
 #convert series to string then join each string and split to create individual words in list   
    string_msg=stringSeries(msg_series)
    joined_string =''
    joined_string=joined_string.join(string_msg)
    list_of_words=joined_string.split()
#remove the and you     
    gen_words=list(STOPWORDS)
    word_list=[i for i in list_of_words if i not in gen_words]
    
    
    return (list_to_df(word_list))
 
    
    #convert list into dataframe then count frequency
def list_to_df(a_list):
    
    if len(a_list)>0:
        
        words_df=pd.DataFrame(a_list)
        value_counts = words_df[0].value_counts()
        words_df=words_df.value_counts().rename_axis('unique_words').reset_index(name='counts')
    #save result of max as df;   
        max_df=words_df[words_df.counts==words_df.counts.max()]
    #some words may have same max count; combine them into one string
        if len(max_df)>1:
            words = ','.join(stringSeries(max_df.unique_words))
        else:
            words = max_df.unique_words[0]

        tot_freq = max_df.counts[0]

    else:
        words=''
        tot_freq=0
    return words, tot_freq
    




In [22]:
#function to convert string series into list, takes series, returns list 
#null values should equal 'Empty

def stringSeries(a_series):
    stringed_s=[s for s in a_series if s!='Empty'] 
    return stringed_s


#redo above function but taking in multiple columns
#cannot work with emoji text as it reads multiple emojis as a different string
#will also return empty as a unique value

def stringCols(a_df):
    stringed_c=np.unique(a_df.values).tolist()
    return stringed_c

In [25]:
#create function to summarize individual records

def singularSum(entire_df):
    single_summary=[]
    members=entire_df.Sender.unique()
#loop through df
    for loop in range (len(members)):
    #create new df for single member
        single_df=entire_df[entire_df['Sender']==members[loop]]
    #create additional columns to calculate days and month names
        single_df['Day']=single_df['Date'].dt.day_name()
        single_df['Month']=single_df['Date'].dt.month_name()
    #create summaries     
        total_messages= single_df.shape[0]
        total_mms=len(single_df[single_df['Chat']==' media shared'])
        freq_day=activeDay(single_df['Day'])
        freq_hour=activeHour(single_df['Time'])
        freq_month=activeDay(single_df['Month'])
        words, high_w, low_w, ave_w = wordCounter(single_df['Chat'])
        freq_word, freq_wc= freqWord(txtMsg(single_df['Chat']))
    



        single_summary.append([members[loop],total_messages,total_mms,freq_day,freq_hour,freq_month, words, 
               high_w, low_w, ave_w, freq_word, freq_wc])    

    summary_df = pd.DataFrame(single_summary, columns = ['Sender','Total Messages','Total MMS','Most Freq Day','Most Freq Hour','Most Freq Month', 'Total Words', 
           'Max Word Count', 'Min Word Count', 'Ave Word Count', 'Most Freq Word', 'Most Freq Word Count'])
    
    return summary_df



    

In [27]:
#supress errors
pd.options.mode.chained_assignment = None  

In [28]:
summary_df=singularSum(whatsapp_df2)

In [29]:
summary_df

,Sender,Total Messages,Total MMS,Most Freq Day,Most Freq Hour,Most Freq Month,Total Words,Max Word Count,Min Word Count,Ave Word Count,Most Freq Word,Most Freq Word Count
0,GC6,97,37,Saturday,21,December,854,246,2,8.80,happy,25
1,GC2,171,3,"Friday, Wednesday",08,June,3423,796,1,20.02,rhapsody,57
2,Uncle,151,7,Wednesday,20,January,2030,705,1,13.44,happy,64
3,Uncle1,49,3,"Friday, Saturday, Thursday",11,May,394,32,1,8.04,happy,32
4,Aunt 4,143,3,Monday,09,May,2280,730,1,15.94,happy,44
5,GC11,103,4,Saturday,09,May,842,162,1,8.17,happy,44
6,GC9,67,3,Friday,"08, 12",August,436,29,1,6.51,happy,23
7,GC7,99,4,Monday,10,May,1218,661,1,12.30,happy,36
8,Aunt1,149,7,Tuesday,09,January,1344,66,1,9.02,happy,44
9,UncleD,50,0,Friday,14,May,1385,642,1,27.70,birthday,22
